In [ ]:
import os

folder_path = "/home/anton/Documents/projects/framework/framework-analysis/data/minimized_run_5/1732116960_yolo11n_160"
file_sizes = {file: os.path.getsize(os.path.join(folder_path, file)) / (1024 * 1024) for file in os.listdir(folder_path)}

sorted_files_by_size = dict(sorted(file_sizes.items(), key=lambda item: item[1]))

sorted_files_by_size

In [ ]:
import pandas as pd
import os

folder_path = "/home/anton/Documents/projects/framework/framework-analysis/data/minimized_run_5/1732116960_yolo11n_160"
worker_dataframes = [pd.read_feather(os.path.join(folder_path, file)) for file in os.listdir(folder_path) if file.startswith('worker') and file.endswith('.feather')]
worker_dataframes[0]

In [ ]:
import re
import pandas as pd
import numpy as np

# Normalize column headers by extracting "instance" and "pid" from the JSON formatted string
def normalize_column_name(col):
    instance_match = re.search(r'"instance":"([^"]+)"', col)
    pid_match = re.search(r'"pid":"([^"]+)"', col)
    # Combine extracted components to form a normalized name
    normalized_name = col.replace(instance_match.group(1) if instance_match else "", "").replace(pid_match.group(1) if pid_match else "", "")
    return normalized_name

# Normalize column names for comparison
normalized_columns = [set(map(normalize_column_name, df.columns)) for df in worker_dataframes]

# Find common normalized columns across all worker dataframes
common_columns = set(normalized_columns[0])
for cols in normalized_columns[1:]:
    common_columns.intersection_update(cols)

# Reconstruct the original column names for selection
actual_common_columns = []
for col in worker_dataframes[0].columns:
    if normalize_column_name(col) in common_columns:
        actual_common_columns.append(col)

# Ensure all workers have the same columns
all_columns = set().union(*[df.columns for df in worker_dataframes])

# Add missing columns with NaN values to all dataframes
filled_worker_dataframes = [
    df.reindex(columns=all_columns, fill_value=np.nan) for df in worker_dataframes
]

# Select only the actual common columns
filled_worker_dataframes = [
    df[actual_common_columns] for df in filled_worker_dataframes
]

# Output the modified first dataframe as an example
filled_worker_dataframes[1]